In [ ]:
import pandas as pd
import numpy as np
# Load your CSV file
df = pd.read_csv('User_data.csv')
# Display the first few rows to inspect the data
# df.head()
df.shape

(2958, 6)

In [ ]:
df['User_rating'] = pd.to_numeric(df['User_rating'], errors='coerce')

In [ ]:
# Drop rows with missing User_rating
df.dropna(subset=['User_rating'], inplace=True)

# Create user-item matrix
user_item_matrix = df.pivot_table(index='Username', columns='Restaurant_names', values='User_rating', fill_value=0)

# Display the first few rows of the user-item matrix
user_item_matrix

Restaurant_names,1 Kitchen Toronto,1-Hour 1000 Islands Cruise from Ivy Lea with views of Boldt Castle,16 Steakhouse,360 The Restaurant at the CN Tower,5-Seen-Fahrt,84 Yesler,9/11 Tribute Museum Admission Ticket,A Dog's Breakfast,A Putia Di Giugio,ALO RESTAURANT,...,Zax,ZazÃ¡ BistrÃ´ Tropical Ipanema,Zees Grill,Zizzi - Hampton Court,Zizzi - Staines,bartaco,cafeteria Litjes,pho metro,querovoar.net,twinOak Wood-Fired Fare
Username,,,,,,,,,,,,,,,,,,,,,
284susanl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Airsucker,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Alan B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Alan T,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Alexander M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sarah_mt_kelly,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
shane cork,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
shropshiretom,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import pandas as pd

# Assuming df is your DataFrame
# For example:
# df = pd.read_csv('your_data.csv')

# Check for null values in the 'User_rating' column
null_values = df['User_rating'].isnull()

# Count the number of null values
null_count = null_values.sum()

print("Number of null values in 'User_rating' column:", null_count)

# Display the rows where 'User_rating' is null
if null_count > 0:
    print("Rows with null values in 'User_rating':")
    print(df[null_values])
else:
    print("No null values in 'User_rating' column.")


Number of null values in 'User_rating' column: 0
No null values in 'User_rating' column.


In [ ]:
pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808327 sha256=a74bfec2965bd76920180f531d0090cf33cd9d0668ae13104bba492b2e88d4a1
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [ ]:
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k

# Assuming df is your DataFrame containing the provided data
# You might need to install lightfm: pip install lightfm

# Create a dataset
dataset = Dataset()
dataset.fit(df['Username'], df['Restaurant_names'])
num_users, num_items = dataset.interactions_shape()

# Build the interactions matrix
(interactions, weights) = dataset.build_interactions(
    (user, restaurant, rating)
    for user, restaurant, rating in zip(df['Username'], df['Restaurant_names'], df['User_rating'])
)

# Build the model
model = LightFM(loss='warp')  # 'warp' stands for Weighted Approximate-Rank Pairwise loss, suitable for implicit feedback

# Re-fit the model using the fit_partial method
model.fit_partial(interactions, epochs=30, num_threads=2)

# Get user and item mappings
user_mapping, _, item_mapping, _ = dataset.mapping()

# Map usernames and restaurant names to indices
user_ids = np.array([user_mapping[user] for user in df['Username']])
restaurant_ids = np.array([item_mapping[restaurant] for restaurant in df['Restaurant_names']])

# Make predictions
scores = model.predict(user_ids, restaurant_ids)

# Evaluate the model (you may need to split your data into training and test sets for a more accurate evaluation)
precision = precision_at_k(model, interactions, k=5).mean()
print(f'Precision at k: {precision}')


Precision at k: 0.6766917109489441


In [ ]:
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import recall_at_k
# Assuming df is your DataFrame containing the provided data
# You might need to install lightfm: pip install lightfm

# Create a dataset
dataset = Dataset()
dataset.fit(df['Username'], df['Restaurant_names'])
num_users, num_items = dataset.interactions_shape()

# Build the interactions matrix
(interactions, weights) = dataset.build_interactions(
    (user, restaurant, rating)
    for user, restaurant, rating in zip(df['Username'], df['Restaurant_names'], df['User_rating'])
)

# Build the model
model = LightFM(loss='warp')  # 'warp' stands for Weighted Approximate-Rank Pairwise loss, suitable for implicit feedback

# Re-fit the model using the fit_partial method
model.fit_partial(interactions, epochs=30, num_threads=2)

# Get user and item mappings
user_mapping, _, item_mapping, _ = dataset.mapping()

# Map usernames and restaurant names to indices
user_ids = np.array([user_mapping[user] for user in df['Username']])
restaurant_ids = np.array([item_mapping[restaurant] for restaurant in df['Restaurant_names']])

# Make predictions
scores = model.predict(user_ids, restaurant_ids)

# Evaluate the model (you may need to split your data into training and test sets for a more accurate evaluation)
precision = precision_at_k(model, interactions, k=5).mean()
print(f'Precision at k: {precision}')

# Evaluate the model (you may need to split your data into training and test sets for a more accurate evaluation)
recall = recall_at_k(model, interactions, k=5).mean()
print(f'Recall at k: {recall}')


Precision at k: 0.7067669034004211
Recall at k: 0.3481053262402921


In [ ]:
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k
from sklearn.metrics import roc_auc_score

# Assuming df is your DataFrame containing the provided data
# You might need to install lightfm: pip install lightfm

# Create a dataset
dataset = Dataset()
dataset.fit(df['Username'], df['Restaurant_names'])
num_users, num_items = dataset.interactions_shape()

# Build the interactions matrix
(interactions, weights) = dataset.build_interactions(
    (user, restaurant, rating)
    for user, restaurant, rating in zip(df['Username'], df['Restaurant_names'], df['User_rating'])
)

# Build the model
model = LightFM(loss='warp')  # 'warp' stands for Weighted Approximate-Rank Pairwise loss, suitable for implicit feedback

# Re-fit the model using the fit_partial method
model.fit_partial(interactions, epochs=30, num_threads=2)

# Get user and item mappings
user_mapping, _, item_mapping, _ = dataset.mapping()

# Map usernames and restaurant names to indices
user_ids = np.array([user_mapping[user] for user in df['Username']])
restaurant_ids = np.array([item_mapping[restaurant] for restaurant in df['Restaurant_names']])

# Make predictions
scores = model.predict(user_ids, restaurant_ids)

# Convert the sparse interactions matrix to a dense numpy array
interactions_dense = interactions.toarray()

# Evaluate the model (you may need to split your data into training and test sets for a more accurate evaluation)
precision = precision_at_k(model, interactions, k=5).mean()
print(f'Precision at k: {precision}')

# Evaluate the model (you may need to split your data into training and test sets for a more accurate evaluation)
recall = recall_at_k(model, interactions, k=5).mean()
print(f'Recall at k: {recall}')





Precision at k: 0.6646616458892822
Recall at k: 0.320025602787073


In [ ]:
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame containing the provided data
# You might need to install lightfm: pip install lightfm

# Create a dataset
dataset = Dataset()
dataset.fit(df['Username'], df['Restaurant_names'])
num_users, num_items = dataset.interactions_shape()

# Build the interactions matrix
(interactions, weights) = dataset.build_interactions(
    (user, restaurant, rating)
    for user, restaurant, rating in zip(df['Username'], df['Restaurant_names'], df['User_rating'])
)

# Split the data into training and test sets
train_interactions, test_interactions = train_test_split(interactions, test_size=0.2, random_state=42)

# Build the model
# model = LightFM(loss='warp')  # 'warp' stands for Weighted Approximate-Rank Pairwise loss, suitable for implicit feedback
model = LightFM(loss='warp', no_components=30, learning_schedule='adagrad')

# Re-fit the model using the fit_partial method on the training set
# model.fit_partial(train_interactions, epochs=30, num_threads=2)
model.fit_partial(train_interactions, epochs=120, num_threads=2)

# Get user and item mappings
user_mapping, _, item_mapping, _ = dataset.mapping()

# Map usernames to indices
user_id = user_mapping['Try2TravelSustainably']  # Replace 'Simon Tremblay' with the target user

# print(user_mapping)

# Map restaurant names to indices for all items
item_ids = np.array([item_mapping[item] for item in df['Restaurant_names']])

# Make predictions for the target user
scores = model.predict(user_id, item_ids)

# Get the indices of the top 5 recommended items
top_items = np.argsort(scores)[::-1][:3]

# Map the indices back to restaurant names
top_restaurants = [item for item, idx in item_mapping.items() if idx in top_items]

# Print the top 5 recommended restaurants
print(f'Top 3 recommended restaurants: {top_restaurants}')


Top 3 recommended restaurants: ['Aqua Sports Center and Scuba Dive', 'Playa Las Gatas', 'Rock n Deli']


In [ ]:
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame containing the provided data
# You might need to install lightfm: pip install lightfm

# Create a dataset
dataset = Dataset()
dataset.fit(df['Username'], df['Restaurant_names'])
num_users, num_items = dataset.interactions_shape()

# Build the interactions matrix
(interactions, weights) = dataset.build_interactions(
    (user, restaurant, rating)
    for user, restaurant, rating in zip(df['Username'], df['Restaurant_names'], df['User_rating'])
)

# Split the data into training and test sets
train_interactions, test_interactions = train_test_split(interactions, test_size=0.2, random_state=42)

# Build the model
model = LightFM(loss='warp', no_components=30, learning_schedule='adagrad')

# Re-fit the model using the fit_partial method on the training set
model.fit_partial(train_interactions, epochs=100, num_threads=2)

# Get user and item mappings
user_mapping, _, item_mapping, _ = dataset.mapping()

# Map the target username to its internal user ID
target_user_id = user_mapping['Try2TravelSustainably']

# Map restaurant names to indices for all items
item_ids = np.array([item_mapping[item] for item in df['Restaurant_names']])

# Make predictions for the target user
target_user_predictions = model.predict(target_user_id, item_ids)

# Get the indices of the top 3 recommended items
top_items = np.argsort(target_user_predictions)[::-1][:3]

# Map the indices back to restaurant names
top_restaurants = [item for item, idx in item_mapping.items() if idx in top_items]

# Print the top 3 recommended restaurants for the target user
print(f'Top 3 recommended restaurants: {top_restaurants}')


Top 3 recommended restaurants: ['Aqua Sports Center and Scuba Dive', "Joe's Hamburgers", 'Rock n Deli']


In [ ]:
pip install scikit-surprise

In [ ]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split

# Assuming you have a DataFrame named df with columns: 'Username', 'Restaurant_names', 'User_rating'
reader = Reader(rating_scale=(1, 100))
data = Dataset.load_from_df(df[['Username', 'Restaurant_names', 'User_rating']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Build the collaborative filtering model
sim_options = {
    'name': 'cosine',  # Use cosine similarity
    'user_based': False  # Item-based collaborative filtering
}
model = KNNBasic(sim_options=sim_options)

# Train the model on the training set
model.fit(trainset)

# Make predictions on the test set
predictions = model.test(testset)

# Get top N restaurant recommendations for each user
from collections import defaultdict

def get_top_n(predictions, n=3):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Sort the predictions for each user and retrieve the top N
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# Get top 5 recommendations for each user
top_n_recommendations = get_top_n(predictions, n=3)

# Print the top 5 recommendations for a specific user (replace 'Simon Tremblay' with the desired username)
username = 'Try2TravelSustainably'
print(f"Top 3 recommendations for {username}: {top_n_recommendations[username]}")

Computing the cosine similarity matrix...
Done computing similarity matrix.
Top 3 recommendations for Try2TravelSustainably: [("Brett's Ice Cream", 41.80626057529611)]


In [ ]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split

# Assuming you have a DataFrame named df with columns: 'Username', 'Restaurant_names', 'User_rating'
reader = Reader(rating_scale=(1, 100))
data = Dataset.load_from_df(df[['Username', 'Restaurant_names', 'User_rating']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Build the collaborative filtering model
sim_options = {
    'name': 'cosine',  # Use cosine similarity
    'user_based': False  # Item-based collaborative filtering
}
model = KNNBasic(sim_options=sim_options)

# Train the model on the training set
model.fit(trainset)

# Make predictions on the test set
predictions = model.test(testset)

# Get top N restaurant recommendations for each user
from collections import defaultdict

def get_top_n(predictions, n=5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Sort the predictions for each user and retrieve the top N
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# Get top 5 recommendations for each user
top_n_recommendations = get_top_n(predictions, n=3)

# Print the top 5 recommendations for a specific user (replace 'Simon Tremblay' with the desired username)
username = 'Try2TravelSustainably'
print(f"Top 5 recommendations for {username}: {top_n_recommendations[username]}")


Computing the cosine similarity matrix...
Done computing similarity matrix.
Top 5 recommendations for Try2TravelSustainably: [("Brett's Ice Cream", 41.80626057529611)]


In [ ]:
from sklearn.metrics import precision_score

# Assuming you have predictions and testset from your model
predictions = model.test(testset)

# Convert predictions to binary (1 if prediction >= threshold, 0 otherwise)
threshold = 3.5  # Set your threshold
binary_predictions = [1 if pred >= threshold else 0 for uid, iid, true_r, pred, _ in predictions]

# Convert true ratings to binary (1 if true rating >= threshold, 0 otherwise)
binary_true_ratings = [1 if true_r >= threshold else 0 for uid, iid, true_r, pred, _ in predictions]

# Calculate precision
precision = precision_score(binary_true_ratings, binary_predictions, average='binary')

print(f'Precision: {precision}')


Precision: 1.0


In [ ]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score

# Assuming you have a DataFrame named df with columns: 'Username', 'Restaurant_names', 'User_rating'
reader = Reader(rating_scale=(1, 100))
data = Dataset.load_from_df(df[['Username', 'Restaurant_names', 'User_rating']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Build the collaborative filtering model
sim_options = {
    'name': 'cosine',  # Use cosine similarity
    'user_based': False  # Item-based collaborative filtering
}
model = KNNBasic(sim_options=sim_options)

# Train the model on the training set
model.fit(trainset)

# Make predictions on the test set
predictions = model.test(testset)

# Set a threshold to convert ratings into binary values (1 for interactions, 0 for non-interactions)
threshold = 3.5
binary_predictions = [1 if pred.est >= threshold else 0 for pred in predictions]

# Extract true ratings
true_ratings = [int(test[2] >= threshold) for test in testset]

# Calculate precision and recall
precision = precision_score(true_ratings, binary_predictions)
recall = recall_score(true_ratings, binary_predictions)

print(f'Precision: {precision}')
print(f'Recall: {recall}')


Computing the cosine similarity matrix...
Done computing similarity matrix.
Precision: 1.0
Recall: 1.0


In [ ]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score

# Assuming you have a DataFrame named df with columns: 'Username', 'Restaurant_names', 'User_rating'
reader = Reader(rating_scale=(1, 100))
data = Dataset.load_from_df(df[['Username', 'Restaurant_names', 'User_rating']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Build the collaborative filtering model
sim_options = {
    'name': 'cosine',  # Use cosine similarity
    'user_based': False  # Item-based collaborative filtering
}
model = KNNBasic(sim_options=sim_options)

# Train the model on the training set
model.fit(trainset)

# Make predictions on the test set
predictions = model.test(testset)

# Experiment with different threshold values
thresholds = [2.5, 3.0, 3.5, 4.0, 4.5]

for threshold in thresholds:
    # Convert ratings into binary values based on the threshold
    binary_predictions = [1 if pred.est >= threshold else 0 for pred in predictions]

    # Extract true ratings
    true_ratings = [int(test[2] >= threshold) for test in testset]

    # Calculate precision and recall
    precision = precision_score(true_ratings, binary_predictions)
    recall = recall_score(true_ratings, binary_predictions)

    print(f'Threshold: {threshold}, Precision: {precision}, Recall: {recall}')


Computing the cosine similarity matrix...
Done computing similarity matrix.
Threshold: 2.5, Precision: 1.0, Recall: 1.0
Threshold: 3.0, Precision: 1.0, Recall: 1.0
Threshold: 3.5, Precision: 1.0, Recall: 1.0
Threshold: 4.0, Precision: 1.0, Recall: 1.0
Threshold: 4.5, Precision: 1.0, Recall: 1.0
